In [2]:
measure_groups = {
    'South America': {
        'providers': {
            'AWS': ['sa-east-1'],
            'AZURE': ['brazilsouth'],
            'GCP': ['southamerica-east1']
        },
        'timezone': 'America/Sao_Paulo'
    },
    'Canada': {
        'providers': {
            'AWS': ['ca-central-1'],
            'AZURE': [],
            'GCP': ['northamerica-northeast1']
        },
        'timezone': 'America/Montreal'
    },
    'East US': {
        'providers': {
            'AWS': ['us-east-1'],
            'AZURE': ['eastus'],
            'GCP': ['us-east4']
        },
        'timezone': 'America/New_York'
    },
    'West US': {
        'providers': {
            'AWS': ['us-west-1', 'us-west-2'],
            'AZURE': ['westus', 'westus2'],
            'GCP': ['us-west2', 'us-west4']
        },
        'timezone': 'America/Los_Angeles'
    },
    'United Kingdom': {
        'providers': {
            'AWS': ['eu-west-2'],
            'AZURE': ['uksouth'],
            'GCP': ['europe-west2']
        },
        'timezone': 'Europe/London'
    },
    'Germany': {
        'providers': {
            'AWS': ['eu-central-1'],
            'AZURE': ['germanywestcentral'],
            'GCP': ['europe-west3']
        },
        'timezone': 'Europe/Berlin'
    },
    'India': {
        'providers': {
            'AWS': ['ap-south-1'],
            'AZURE': ['centralindia'],
            'GCP': ['asia-south1']
        },
        'timezone': 'Asia/Kolkata'
    },
    'Japan': {
        'providers': {
            'AWS': ['ap-northeast-1'],
            'AZURE': ['japaneast'],
            'GCP': ['asia-northeast1']
        },
        'timezone': 'Asia/Tokyo'
    },
    'Australia': {
        'providers': {
            'AWS': ['ap-southeast-2'],
            'AZURE': ['australiaeast'],
            'GCP': ['australia-southeast1']
        },
        'timezone': 'Australia/Sydney'
    }
}

In [3]:
import pandas as pd
from multiprocessing import  Pool
import multiprocessing
import datetime

from functools import partial
import numpy as np

In [4]:
dataset = pd.read_parquet('raw-dataset.parquet').convert_dtypes()

In [5]:
# Turn dataset into datetime:
dataset['driver_invocation'] = pd.to_datetime(
    dataset['driver_invocation'], format='%Y%m%dT%H%M%S%f')
dataset['workload_invocation'] = pd.to_datetime(
    dataset['workload_invocation'], format='%Y%m%dT%H%M%S%f')

In [6]:
# Cut Off Data
data_from = datetime.datetime(year=2021,month=8,day=1)
dataset = dataset[dataset['driver_invocation'] >= data_from]

In [7]:
# Preprocessing based on utc:
dataset['dow_utc'] = dataset['driver_invocation'].dt.day_name()
dataset['tod_utc'] = dataset['driver_invocation'].dt.strftime('%H%M')

In [8]:
# Localize Timezones for driver invocation
for mg in measure_groups:
    regions = []
    for provider in measure_groups[mg]['providers']:
        regions.extend(measure_groups[mg]['providers'][provider])

    dataset.loc[(dataset['region'].isin(regions)),
                'timezone'] = measure_groups[mg]['timezone']
    dataset.loc[(dataset['region'].isin(regions)), 'measure group'] = mg

In [9]:
def get_local_dow_of_the_week(df):
    ts = df['driver_invocation']
    tz = df['timezone']
    return  ts.tz_localize('utc').tz_convert(tz).day_name()

def get_local_tod_of_the_week(df):
    ts = df['driver_invocation']
    tz = df['timezone']
    return ts.tz_localize('utc').tz_convert(tz).strftime('%H%M')

def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(func, data_subset):
    return data_subset.apply(func, axis=1)

def parallelize_on_rows(data, func):
    return parallelize(data, partial(run_on_subset, func), multiprocessing.cpu_count())

In [10]:
dataset['local_dow'] = parallelize_on_rows(dataset[['driver_invocation', 'timezone']], get_local_dow_of_the_week) 
dataset['local_tod'] = parallelize_on_rows(dataset[['driver_invocation', 'timezone']], get_local_tod_of_the_week) 

In [11]:
dataset[['local_dow', 'local_tod']] = dataset[['local_dow', 'local_tod']].astype('category')

In [12]:
dataset.to_parquet("dataset-step-01.parquet", index=False, engine="pyarrow")

In [ ]:
dataset = pd.read_parquet("dataset-step-01.parquet")

In [ ]:
dataset.memory_usage().sum()

In [ ]:
dataset = dataset.convert_dtypes()

In [ ]:
custom_dtypes = {
    'SAAFMemoryDeltaError': 'category',
    'SAAFMemoryError': 'category',
    'vendorId': 'category',
    'platform': 'category',
    'payload': 'category',
    'functionRegion': 'category',
    'linuxVersion': 'category',
    'lang': 'category',
    'functionName': 'category',
    'cpuType': 'category',
    'provider': 'category',
    'timezone': 'category',
    'region': 'category',
    'dow_utc': 'category',
    'tod_utc': 'category',
    'measure group': 'category',
    'cpuModel': 'category',
    '2_thread_id': 'category',
    '1_run_id': 'category',
    'newcontainer': 'bool',
    'platform': 'category',
    'version': 'category',
    'containerID': 'category',
}

In [ ]:
for column, dtype in custom_dtypes.items():
    dataset[column] = dataset[column].astype(dtype)

In [ ]:
for col in dataset.columns:
    print(f"--- {col} ---")
    print(dataset[col].dtypes)
    print(dataset[col].describe())
    print(dataset[col].memory_usage())

In [ ]:
# Drop Columns
drop_cols = ['message', 'payload']
dataset = dataset.drop(drop_cols, axis = 1)

In [ ]:
dataset.columns

In [ ]:
dataset.dtypes

In [ ]:
dataset.memory_usage().sum()

In [ ]:
dataset.to_parquet("dataset-compressed.parquet", index=False, engine="pyarrow")

In [ ]:
# 17379823784
# 15807614178
# 19158897503